In [29]:
HUGGINGFACE_API_KEY=""
GROQ_API_KEY=""

In [30]:
import os
import asyncio
import getpass 
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb

In [31]:
os.environ['HUGGINGFACE_API_TOKEN']=HUGGINGFACE_API_KEY
os.environ['GROQ_API_KEY']=GROQ_API_KEY

In [32]:
from langchain_groq import ChatGroq

In [33]:
llm=ChatGroq(model_name="llama-3.3-70b-versatile")

In [34]:
from langchain.vectorstores import FAISS
import faiss 
import pandas as p

In [35]:
embeddings=HuggingFaceEmbeddings()

C:\Users\Aarav Sharma\AppData\Local\Temp\ipykernel_6080\1837158031.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()


In [36]:
policy_df = p.read_csv("sbilife_policies_4.csv", encoding="ISO-8859-1")
texts = policy_df.apply(lambda row: " | ".join(row.astype(str)), axis=1).tolist()

In [37]:
policy_vectorstore=FAISS.load_local("policy_faiss_index",embeddings,allow_dangerous_deserialization=True)

In [38]:
profile_df=p.read_csv("SBI_Life_Policyholders_10000.csv")


In [39]:
texts=profile_df.apply(lambda row: " | ".join(row.astype(str)), axis=1).tolist()

In [40]:
profile_vectorstore=FAISS.load_local("profile_faiss_index",embeddings,allow_dangerous_deserialization=True)

In [41]:
from langchain.chains import RetrievalQA,LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [42]:
manual_template="""
Suggest similar SBI Life policies on the basis of the input taken from the user and the profiles having similar background as the user.
The user data is also provided below:
Name:{name},
Age:{age},
Occupaion:{occupation},
Education:{education},
Annual Income:{income}
Provide complete answers with the name of the policies being suggested, their key benefits, the type of policy, the url, anuual premium range and entry age. Suggest only the relavant policies.
"""
prompt_manual = PromptTemplate(
    input_variables=["name","age","occupation","education","income"],
    template=manual_template,
)

In [43]:
from langchain.vectorstores.base import VectorStoreRetriever
class AsyncMultiVectorRetriever():
    def __init__(self, retrievers):
        """Initialize with multiple retrievers (vector stores)."""
        self.retrievers = retrievers

    async def get_relevant_documents(self, query):
        """Fetch relevant documents from all vector stores."""
        docs = []
        tasks = [asyncio.to_thread(retriever.get_relevant_documents, query) for retriever in self.retrievers]
        results = await asyncio.gather(*tasks)
        return [doc for docs in results for doc in docs]
multi_retriever = AsyncMultiVectorRetriever(retrievers=[profile_vectorstore, policy_vectorstore])

In [44]:
chatbot_template="""
You are a friendly, conversational SBI Life policy recommendation assistant that helps customers find SBI Life policies that match their profile and background information.
This includes their occupation, education, annual income, their existing policies and most important of all for what purpose they need a policy. 
From the following context and chat history, assist customers in finding what they are looking for based on their input. 
For each question, suggest three policies, including their name, type, key benefits, annual premium range, entry age and url.
Similar orthe same profile as the user will also be retrieved. Using these profiles make your suggestions as well. 
If you don't have any policy to suggest, then just say you don't have any policy to suggest as per the requirements, don't try to make up an answer.

{context}

chat history:{history}

input:{question}

Your response:

"""

In [45]:
chatbot_prompt=PromptTemplate(input_variables=["context","history","question"],template=chatbot_template)
memory = ConversationBufferMemory(memory_key="history", input_key="question", return_messages=True)

In [46]:
qa_chain = LLMChain(llm=llm, prompt=chatbot_prompt, verbose=True)
#while True:
    #query = input("\nUser: ")  
    #if query.lower() in ["exit", "quit", "bye"]:
        #print("\nChatbot: Goodbye!")
        #break  
    #retrieved_docs = retrieve_documents(query)
    #retrieved_text = "\n".join([doc.page_content for doc in retrieved_docs])
    #response = qa.run(context=retrieved_text)  
    #response = qa_chain.run(context=multi_retriever, history=memory.load_memory_variables({})["history"], question=query)
    #memory.save_context({"question": query}, {"response": response})
    #print("\nChatbot:", response)

In [47]:
async def async_chat_loop():
    while True:
        query = input("\nUser: ")
        if query.lower() in ["exit", "quit", "bye"]:
            print("\nChatbot: Goodbye!")
            break
        chat_history = await asyncio.to_thread(memory.load_memory_variables, {})  
        response = await asyncio.to_thread(
            qa_chain.run,
            context=multi_retriever, 
            history=chat_history["history"],
            question=query
        )
        await asyncio.to_thread(memory.save_context, {"question": query}, {"response": response})

        print("\nChatbot:", response)


In [48]:
async def main():
    print("Welcome to SBI Life Policy Assistant! Type 'exit' to quit.\n")
    await async_chat_loop()  # Run chatbot asynchronously


In [49]:
try:
    import nest_asyncio
    nest_asyncio.apply()  # Allow nested event loops
    await main()  # Directly await the main function
except RuntimeError:
    # If no event loop is running, use asyncio.run()
    asyncio.run(main())

Welcome to SBI Life Policy Assistant! Type 'exit' to quit.


User: exit

Chatbot: Goodbye!
